In [1]:
import requests
import pandas
from urllib.parse import quote

# API CALL with multiple games

In [14]:


import requests
import json
from urllib.parse import quote

# Steam API URLs
REVIEW_URL = "https://store.steampowered.com/appreviews/"
STORE_URL = "https://store.steampowered.com/api/appdetails?appids="

# List of Game IDs (Example: Add more game IDs here)
games = [
    1677280, 570, 730, 578080, 440, 271590, 252490, 620, 394690, 1328670,
    275850, 1817070, 413150, 271590, 1281930, 578080, 227300, 381210, 892970, 
    526870, 1085660, 359550, 1454400, 292030, 582010, 374320, 236850, 381210,
    289070, 1091500, 1466860, 1174180, 377160, 227300, 244850, 548430, 239140,
    1245620, 583950, 678960, 359320, 823500, 2050650, 578080, 250900, 1237970,
    493340, 813780, 620980, 638970, 282800, 310560, 236430, 1084600, 239160,
    1589560, 234650, 359870, 291550, 282800, 408410, 105600, 668580, 640820,
    362490, 636480, 1517290, 739630, 1018020, 1454400, 1030840, 739630, 813780,
    554920, 897450, 1238860, 292030, 678960, 739630, 814380, 892970, 236850, 
    431240, 1238860, 218620, 239140, 550650, 602960, 1187140, 1446780, 216150, 
    1222670, 552520, 2399830, 2842040, 2767030, 3159330, 346110, 1158310, 2878980, 
    993090, 298110, 582160, 588650, 1911610, 2756920, 3123410, 2232880, 1940340, 1159420, 
    2183900, 1726120, 2001120, 2943650, 1771300, 1086940
]

games = [str(game_id) for game_id in games]

# API Parameters
APPENDIX = "?json=1&language=english"
CURSOR_APPENDIX = "&cursor="

# Store all reviews
newlist = []

# Function to get game details (Name & Genre)
def get_game_details(game_id):
    response = requests.get(STORE_URL + game_id)
    if response.status_code == 200:
        game_data = response.json()
        if game_id in game_data and "data" in game_data[game_id]:
            game_info = game_data[game_id]["data"]
            game_name = game_info.get("name", "Unknown Game")
            genres = [genre["description"] for genre in game_info.get("genres", [])]
            genre = ", ".join(genres) if genres else "Unknown Genre"
            return game_name, genre
    return "Unknown Game", "Unknown Genre"

# Loop through multiple games
for game_id in games:
    print(f"Scraping reviews for game ID: {game_id}")

    # Get game details
    game_name, genre = get_game_details(game_id)

    # Reset cursor and result list for each game
    cursor = "*"
    result = []

    # Fetch up to 5 pages of reviews per game
    for i in range(20):
        url2 = REVIEW_URL + game_id + APPENDIX + "&language=english" + CURSOR_APPENDIX + cursor
        print(url2)
        response = requests.get(url2)

        if response.status_code == 200:
            reviews_data = response.json()
        else:
            print(f"Error {response.status_code} for game ID {game_id}")
            break

        try:
            cursor = quote(reviews_data["cursor"])
        except KeyError:
            result.append(reviews_data)
            break
        result.append(reviews_data)

    # Store reviews with game details
    for i in result:
        if "reviews" in i:
            for j in i["reviews"]:
                temp_list = [game_name, j["review"], j["voted_up"], genre]
                newlist.append(temp_list)

# Print sample output
print("\nExample Review Data:")
for item in newlist[:5]:  # Show first 5 reviews
    print(item)


Scraping reviews for game ID: 1677280
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=*
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFZz1GAAAAB79/HSBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFaAAAAAAAB53MTQBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFaCXKAAAAB8vZHQBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFaePmAAAAB7wcnTBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFavdWAAAAB08obQBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=AoIFQFbbT2AAAAB/75HMBQ%3D%3D
https://store.steampowered.com/appreviews/1677280?json=1&language=english&language=english&cursor=Ao

In [15]:
df = pandas.DataFrame(newlist, columns = ["Game", "Review","Recommended", "Genre"])
#df["Genre"] = "Strategy"
df

,Game,Review,Recommended,Genre
0,Company of Heroes 3,I still believe CoH2 is the superior choice cu...,False,"Action, Strategy"
1,Company of Heroes 3,I Pre-ordered the premium edition version of t...,False,"Action, Strategy"
2,Company of Heroes 3,"An underwhelming, expensive cash grab. Very lo...",False,"Action, Strategy"
3,Company of Heroes 3,Here is my actual review. COH3 has the best Qo...,True,"Action, Strategy"
4,Company of Heroes 3,They listed my GPU as supported and then quiet...,False,"Action, Strategy"
...,...,...,...,...
40644,Baldur's Gate 3,It's not just a game! It's a whole story that ...,True,"Adventure, RPG, Strategy"
40645,Baldur's Gate 3,It's the closest you're going to get to playin...,True,"Adventure, RPG, Strategy"
40646,Baldur's Gate 3,verry gooood gammeeee. I slept on it for to lo...,True,"Adventure, RPG, Strategy"
40647,Baldur's Gate 3,Funny wild magic sorceror Durge run made every...,True,"Adventure, RPG, Strategy"


In [16]:
df.to_csv("Steam_Reviews.csv", mode = "a", index = False)

In [20]:
# Count positive (True) reviews
positive_count = df["Recommended"].sum()

# Count negative (False) reviews
negative_count = len(df) - positive_count  # Total reviews minus positive reviews

print(f"Positive Reviews: {positive_count}")
print(f"Negative Reviews: {negative_count}")
print("Genres:", df["Genre"].nunique())


Positive Reviews: 34045
Negative Reviews: 6604
Genres: 53
